# 불법주정차 데이터 위도/경도 수집 (Kakao Api)
- https://developers.kakao.com
- 카카오맵 api 허용 설정 필수

### Present working directory

In [12]:
!pwd

/Users/lyuhoyun/Documents/GitHub/Data-projects/dacon_sri_suwon_data_analysis


### 패키지 설치

In [11]:
!pip install pandas numpy tqdm requests

### 패키지 다운로드

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import time
import json
import requests

### 카카오 API키 및 API 동작확인

In [ ]:
api_key = "API KEY"

In [4]:
addr = '서울시 송파구 위례광장로 185'
url = 'https://dapi.kakao.com/v2/local/search/address.json?query={address}'.format(address=addr)
headers = {
    "Authorization": f"KakaoAK {api_key}"
}
params = {"query": addr}

result = json.loads(str(requests.get(url, headers=headers).text))
display(result)

{'documents': [{'address': {'address_name': '서울 송파구 장지동 878',
    'b_code': '1171010900',
    'h_code': '1171064700',
    'main_address_no': '878',
    'mountain_yn': 'N',
    'region_1depth_name': '서울',
    'region_2depth_name': '송파구',
    'region_3depth_h_name': '위례동',
    'region_3depth_name': '장지동',
    'sub_address_no': '',
    'x': '127.14004016241',
    'y': '37.4804774688545'},
   'address_name': '서울 송파구 위례광장로 185',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '서울 송파구 위례광장로 185',
    'building_name': '위례신도시 송파푸르지오',
    'main_building_no': '185',
    'region_1depth_name': '서울',
    'region_2depth_name': '송파구',
    'region_3depth_name': '장지동',
    'road_name': '위례광장로',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '127.14004016241',
    'y': '37.4804774688545',
    'zone_no': '05848'},
   'x': '127.14004016241',
   'y': '37.4804774688545'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

### 데이터 불러오기

In [5]:
data1 = pd.read_csv('parking_notcctv_1.csv')
data2 = pd.read_csv('parking_notcctv_2.csv')
data3 = pd.read_csv('parking_notcctv_3.csv')
data4 = pd.read_csv('parking_notcctv_4.csv')
data5 = pd.read_csv('parking_notcctv_5.csv')

### 데이터 수집 및 적용 함수

In [11]:
def get_lat_lng(addr):
    headers = {
        "Authorization": f"KakaoAK {api_key}"
    }
    params = {"query": addr}
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    
    response = requests.get(url, headers=headers, params=params)
    time.sleep(0.15)
    if response.status_code == 200:
        documents = response.json().get("documents")
        if documents:
            return float(documents[0]["y"]), float(documents[0]["x"])  # 위도, 경도
    return None, None


def add_to_pos(data):
    tqdm.pandas()

    latitudes = []
    longitudes = []

    for i,(_, row) in enumerate(tqdm(data.iterrows(), total=len(data))):
        if i>0 and i%1000 == 0:
            print(f'{i} rows. 중간 딜레이 5se\n')
    
        
        lat,lng = get_lat_lng(row['단속장소'])
        latitudes.append(lat)
        longitudes.append(lng)

    data['latitude'] = latitudes
    data['longitude'] = longitudes

    #display(data.head(10))
    return data

In [9]:
#샘플 테스트
sample=data1.sample(10)

### 적용

In [12]:
add_to_pos(sample)

100%|██████████| 10/10 [00:02<00:00,  4.74it/s]


,집계년도,시군명,관리기관명,단속일시정보,단속장소,단속방법,데이터기준일자,단속일시_월,단속일시_연,단속일시_일,단속일시_요일,단속일시_시,단속일시_isdaytime,단속일시_season,latitude,longitude
19623,2022,수원시,팔달구청,2022-02-12 12:56:00,인계동 1123-18 (인도),국민신문고,2023-12-01,2,2022,12,Saturday,12,morning,winter,37.262203,127.033793
88457,2022,수원시,팔달구청,2022-06-16 15:04:00,경기도수원시팔달구지동 수원시 팔달구 지동 444,보행,2023-12-01,6,2022,16,Thursday,15,afternoon,summer,37.276367,127.019671
28136,2022,수원시,영통구청,2022-03-02 05:01:00,영통구 매탄동 1301,국민신문고,2023-12-01,3,2022,2,Wednesday,5,night,spring,37.262654,127.046426
12021,2022,수원시,팔달구청,2022-01-24 20:21:00,경기도수원시팔달구화서동 수원시 팔달구 화양로 38번길 135-10삼양연립,보행,2023-12-01,1,2022,24,Monday,20,night,winter,37.281593,127.004137
7139,2022,수원시,팔달구청,2022-01-14 14:57:00,화서동 화서동 60-7(횡단보도),국민신문고,2023-12-01,1,2022,14,Friday,14,afternoon,winter,37.279867,127.006438
46314,2022,수원시,팔달구청,2022-04-04 14:44:00,인계동 인계동 세지로188,주행형,2023-12-01,4,2022,4,Monday,14,afternoon,spring,37.268049,127.022639
21722,2022,수원시,장안구청,2022-02-16 15:10:00,천천동 만석로5번길,주행형,2023-12-01,2,2022,16,Wednesday,15,afternoon,winter,37.295971,126.982079
84742,2022,수원시,팔달구청,2022-06-10 19:03:00,인계동 965-18 (횡단보도),국민신문고,2023-12-01,6,2022,10,Friday,19,night,summer,37.270409,127.026629
31142,2022,수원시,영통구청,2022-03-07 16:20:00,영통구 하동 1044-2,국민신문고,2023-12-01,3,2022,7,Monday,16,afternoon,spring,37.286947,127.063850
97563,2022,수원시,팔달구청,2022-07-03 20:21:00,화서동 96-23 (교차로모퉁이 5m 이내),국민신문고,2023-12-01,7,2022,3,Sunday,20,night,summer,37.277614,127.002560


### 실제 데이터 주소 -> 위/경도 전환 
시간 소요될 수 있음. 요청 당 0.15초 딜레이

In [ ]:
partial_data = data1

In [ ]:
add_to_pos(partial_data)

In [ ]:
partial_data.to_csv('data_by_who', index=False)